In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import os
import glob
import matplotlib.pyplot as plt
import math
import cv2


%matplotlib inline

In [2]:
def resize(img,size):
    """size is a tuple"""
    """ returns resized images"""
    return cv2.resize(img,size)

def to_hsv(img):
    """return image in HSV space"""
    return cv2.cvtColor(img,cv2.COLOR_RGB2HSV)

def to_gray(img):
    """return image in gray space"""
    return cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

def sum(arr):
    """returns sum and no. of pixels between 20 and 240"""
    sum = 0
    count = 0
    for i in arr:
        for j in i:
            if(j>20 and j<240): #only pixels whose value is between 20 and 240
                sum+=j
                count+=1

    return (sum,count)

def pooling(image, pool_size, code, padding):
    """
    different codes for different pooling
    code min :min pooling
    code max :max pooling 
    code mean :mean pooling 
    code std :standard deviation pooling
    returns a image with padding operation and pooling operation
    """

    padded = arr = np.zeros((image.shape[0] + padding*2, 
                       image.shape[1] + padding*2))
    
    #  inserting image into zero array
    padded[int(padding):-int(padding), int(padding):-int(padding)] = image
    
    
    # print(f'original image size: {image.shape}')
    # print(f'padded image size: {padded.shape}')

    input_height, input_width = padded.shape
    pool_height, pool_width = pool_size

    # Calculate the output dimensions
    output_height = input_height - pool_height + 1
    output_width = input_width - pool_width + 1

    # Initialize the output data
    output_data = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            # Extract the region of interest (ROI)
            roi = padded[i : i + pool_height, j : j + pool_width]
            
            if code=='min':
                # Apply min pooling within the ROI
                output_data[i, j] = np.min(roi)

            if code=='max':
                # Apply max pooling within the ROI
                output_data[i, j] = np.max(roi)

            if code=='mean':
                # Apply mean pooling within the ROI
                output_data[i, j] = np.mean(roi)

            if code=='std':
                # Apply min pooling within the ROI
                output_data[i, j] = np.std(roi)


    # print(f'{code} pooled image size: {output_data.shape}')
    return output_data

In [3]:
def feature(data):
    """Return all the 12 features as a numpy array"""
    uid,img,label = data
    img = resize(img,(250,250))

    #RGB SPACE
    r, g, b = cv2.split(img)
    sum_img = [sum(r),sum(g),sum(b),sum(r-g)]
    mean_features = [i[0]/i[1] for i in sum_img]
    mean_r,mean_g,mean_b,mean_rg = mean_features
    # 4 features done in RGB SPACE

    
    #HSV SPACE
    hsv = to_hsv(img)
    h,s,v = cv2.split(hsv)
    h = h/h.max()
    nH = np.count_nonzero(h>0.95)
    HHR = nH/h.size
    # HHR found

    
    #GRAY SPACE
    gray = to_gray(img)
    B_sum, B_size = sum(gray)
    B = B_sum/B_size # FOUND B

    #ENTROPY in gray space
    eq = cv2.equalizeHist(gray)
    unique, counts = np.unique(eq, return_counts=True)
    #only pixels whose value is between 20 and 240
    total_counts = counts[21:240].sum() 
    Ent = np.sum(np.array([-i*(i/total_counts)*math.log((i/total_counts),2) for i in counts[21:240]])) #Found Entropy

    #Calculating the 'G' features
    Ixy = gray
    min_Ixy = pooling(image=Ixy, pool_size=(3,3), code='min', padding=1)
    max_Ixy = pooling(image=Ixy, pool_size=(3,3), code='max', padding=1)
    mean_Ixy = pooling(image=Ixy, pool_size=(3,3), code='mean', padding=1)
    std_Ixy = pooling(image=Ixy, pool_size=(3,3), code='std', padding=1)
    
    g1 = Ixy - min_Ixy
    g2 = max_Ixy - Ixy
    g3 = Ixy - mean_Ixy
    g4 = std_Ixy
    g5 = Ixy
    
    G1 = g1.sum()/g1.size
    G2 = g2.sum()/g2.size
    G3 = g3.sum()/g3.size
    G4 = g4.sum()/g4.size
    G5 = g5.sum()/g5.size

    feature_all = [uid,mean_r,mean_g,mean_b,mean_rg,HHR,Ent,B,G1,G2,G3,G4,G5,label]
    return feature_all

In [9]:
mask_left_palm_files = glob.glob("../../Field Data/Individual Anatomy/Galaxy M34/Left Palm_files/left_palm_segmasks/SegmentationClass/*.png")
left_palm_path = "../../Field Data/Individual Anatomy/Galaxy M34/Left Palm_files/Images_left_palm\\"

In [10]:
left_palm_img_list = []
for mask_name in mask_left_palm_files:
    mask = mask_name.split("\\")[-1].split(".")[0] + ".jpg"
    left_palm_img_list.append(mask)

In [11]:
# df = pd.DataFrame(left_palm_img_list, columns=["Annotated_Data"])
# df.to_csv('annotations_left_palm.csv', index=False)

In [13]:
left_palm_truth = pd.read_csv("annotations_left_palm.csv")
left_palm_truth.head(5)

,Annotated_Data,Avaiable ID,Hb Value,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1709617238375.jpg,202403051,11.5,NaN,NaN,Average,11.53193
1,1709617534838.jpg,202403052,11.6,NaN,NaN,NaN,NaN
2,1709618528581.jpg,202403056,11.1,NaN,NaN,NaN,NaN
3,1709619358067.jpg,202403058,12.1,NaN,NaN,NaN,NaN
4,1709619811692.jpg,202403059,12.2,NaN,NaN,NaN,NaN


In [15]:
Total_Data = []
for mask_path in mask_left_palm_files:
    name = mask_path.split("\\")[-1].split(".")[0] + ".jpg"
    
    mask_bgr = cv2.imread(mask_path)
    img_bgr = cv2.imread(left_palm_path + name)
    mask = cv2.cvtColor(mask_bgr,cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(img_bgr,cv2.COLOR_BGR2RGB)

    roi = cv2.bitwise_and(img, img, mask = mask)

    uid = left_palm_truth[left_palm_truth['Annotated_Data'] == name]["Avaiable ID"].tolist()[0]

    label = left_palm_truth[left_palm_truth['Annotated_Data'] == name]["Hb Value"].tolist()[0]

    data_sent = [uid,roi,label]

    features = feature(data_sent)

    Total_Data.append(features)

In [16]:
cols = ['number','mean_r','mean_g','mean_b','mean_rg','HHR','Ent','B','G1','G2','G3','G4','G5','label']

complete_data = pd.DataFrame(Total_Data,columns=cols)

In [17]:
complete_data.head()

,number,mean_r,mean_g,mean_b,mean_rg,HHR,Ent,B,G1,G2,G3,G4,G5,label
0,202403051,172.297381,97.355683,80.445888,80.970671,0.011872,1876.350134,118.767315,6.513328,5.938976,1.728040e-17,4.259915,42.591632,11.5
1,202403052,181.642334,109.729594,92.693423,71.993217,0.000608,2855.828483,129.305617,3.405328,3.360352,1.818989e-17,2.424010,48.548048,11.6
2,202403056,169.608618,100.018009,87.862148,69.718959,0.001088,1209.532432,119.406123,4.586608,4.052848,4.547474e-19,2.911402,27.709904,11.1
3,202403058,145.644678,105.351410,101.135942,60.917965,0.066928,1303.436750,119.787072,5.408208,4.911472,-2.432898e-17,3.614251,40.254032,12.1
4,202403059,188.322497,141.513432,138.821386,46.932536,0.092448,2197.192470,155.207958,4.848720,4.613312,3.770667e-03,3.281453,51.865184,12.2


In [18]:
# complete_data.to_csv('left_palm_data.csv',index = False)